In [1]:
import numpy as np
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM, SpatialDropout1D, Dropout, Flatten
#from keras.datasets import imdb
import pandas as pd
import numpy as np
#устанавливаем seed
np.random.seed(42)
import pymorphy2
import re
import sklearn.feature_extraction.text as fex


Using TensorFlow backend.


In [2]:
prodIdColName = "prodId"
subColName = "SubName"
sentColName = "sentence"
trainDF = pd.DataFrame({prodIdColName: [1, 1, 1,2], subColName: ['телефон', 'смартвон', 'жизнь', 'жизнь'], sentColName:['SENTтелефон', 'SENTсмартвон', 'SENT00жизнь', 'SENT2жизнь'] })

In [3]:
testDF = pd.DataFrame({prodIdColName: [3, 3, 3,4], subColName: ['телефон', 'смартвон', 'жизнь', 'жизнь'], sentColName:['3SENTтелефон', '3SENTсмартвон', '3SENT00жизнь', '3SENT2жизнь'] })

In [4]:
full = trainDF.append(testDF)

In [5]:
df_train = pd.read_csv('dataset1.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)


In [6]:
textColName = "TEXT"
negatColName =  "DRAWBACKS"
positColName = "BENEFITS"
ratingColName = "RATING"
catColName = "CATEGORY_ID"
brandColName = "BRAND_ID"
catNameColName= "CATEGORY_NAME"
tovarIdColName = "PRODUCT"
df_train.head()

,Unnamed: 0,PRODUCT,CATEGORY_ID,BRAND_ID,CATEGORY_NAME,BRAND_NAME,TYPE,SUBJECT,TEXT,POSTED_DATE,RATING,BENEFITS,DRAWBACKS,RECOMMENDED,LIKES_COUNT,DISLIKES_COUNT
0,0,30024724,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,Что особенно ценно - при профилактике принтера...,23.10.16 22:26:00,5.0,"хорошая скорость печати, двусторонняя печать, ...","Рядом ""на стол"" не поставишь, место много зани...",1.0,2.0,0.0
1,1,30024724,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,Он у нас три месяца в офисе стоит уже - большо...,17.04.16 11:17:42,5.0,NaN,NaN,NaN,0.0,2.0
2,2,30024725,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,"Просто небо и земля с тем принтером, который у...",15.04.16 20:22:47,4.0,NaN,NaN,NaN,0.0,1.0
3,3,30024727,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,С такой вроде бы навороченной техникой все смо...,У нас в кабинете недавно его поставили. Первое...,18.04.16 20:01:57,5.0,NaN,NaN,NaN,0.0,0.0
4,4,30024727,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,NaN,Очень многофункциональная штука. Есть все нео...,23.04.16 12:03:39,5.0,NaN,NaN,NaN,1.0,0.0


In [7]:
morph = pymorphy2.MorphAnalyzer()

def normalize(text):
    return ' '.join([morph.parse(str(w))[0].normal_form for w in text.split()])
def clean_text(text):
    text = str(text)
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-\'>><`', ' ', text) #deleting symbols  
    text = re.sub('-', ' ', text) #deleting symbols  
    #text = ' '.join(word[:] for word in text.split() if len(word)>3)
    #text = text.encode("utf-8")
    return text 

In [8]:
#df_train[textColName]=df_train[textColName].apply(clean_text).apply(normalize)

In [12]:
#import tparser
from tparser import tomita
from tparser.tomita_parser import text_parse
import pandas as pd

In [13]:
catidNameForBuild = "SMARTPHONES"
category_data = testAllCut[testAllCut[catNameColName].str.contains(catidNameForBuild)==True]

NameError: name 'testAllCut' is not defined

In [ ]:
textAndProd = category_data.ix[:,[textColName,tovarIdColName]]
textAndProdGrS = textAndProd.groupby(tovarIdColName )[tovarIdColName, textColName].agg(lambda x: "{%s}" % ', '.join(x)).reset_index()
print(len(textAndProd))
print(len(textAndProdGrS))

In [ ]:
tfidf = fex.TfidfVectorizer()

In [ ]:
tfIdfMatrix = tfidf.fit_transform(textAndProdGrS[textColName])
ftNames = tfidf.get_feature_names()
